# RHESSys Workflow Using Gage Identification

RHESSysWorkflows provides a series of Python tools for performing [RHESSys](https://github.com/RHESSys/RHESSys) data preparation workflows. These tools build on the workflow system defined by [EcohydroLib](https://github.com/selimnairb/EcohydroLib) and [RHESSysWorkflows](https://github.com/selimnairb/RHESSysWorkflows).


**Note:** Users interested in seeing step outputs, remove **output = ** from the command line. Some output steps have been hidden, as the output lengths can be long. <br>
<br>
<br>

This cell imports the necessary python libraries for the notebook 

In [ ]:
import os
import logging
from utilities import hydroshare
import shutil
from utilities.rhessys import *
%matplotlib inline

print 'Finished Step'

The `RHESSysWorkflow` class is provided as part of the `rhessys_wf` library (imported above) to streamline your interaction with the `RHESSysWorkflows` [core functions](https://github.com/selimnairb/RHESSysWorkflows).  More information about this class can be obtained by executing the `help(RHESSysWorkflow)` command. <br>
<br>

Create an instance of the `RHESSysWorkflow` class and assign it to the variable `w`, using <br>

* USGS gage **01589312** [(DEAD RUN NEAR CATONSVILLE, MD)](http://waterdata.usgs.gov/usa/nwis/uv?01589312) , <br>
* a start date of **2008-01-01**, and <br>
* an end date of **2010-01-01**.<br>

This command will create a clean directory for the given project name of **myRHESSysGageSimulation** in the JupyterHub's default data directory. **If the directory exists, you will be prompted to use, or overwrite.** <br>

In [ ]:
w = RHESSysWorkflow(project_name='myRHESSysGageSimulation', 
                    gageid='01589312',
                    start_date='2008-01-01',
                    end_date='2010-01-01'
                    )

print 'Finished Step'

The `RHESSysWorkflow` class uses a logging library to document output and errors.  We can display these messages in our notebook by attaching to the logger and redirecting the output to stdout.  While this step is not necessary, it will provide us with more verbose output.

In [ ]:
# Get root logger for RHESSysWorkflow and set the output to stdout
root_logger = logging.getLogger()
root_logger.addHandler(logging.StreamHandler(sys.stdout))

print 'Finished Step'

Using the watershed parameters defined within the `RHESSysWorkflow object` (***w***), `get_NHDStreamflowGageIdentifiersAndLocation` retrieves NHDPlus2 streamflow gage identifiers (reachcode, measure along reach in percent) for a USGS gage. <br>
<br>
The function `get_CatchmentShapefileForNHDStreamflowGage` generates a shapefile for the drainage area of an NHDPlus2 streamflow gage using web services from [Horizon Systems NHDPlus Version 2](http://www.horizon-systems.com/NHDPlus/NHDPlusV2_home.php). <br><br>
The `get_BoundingboxFromStudyareaShapefile` function calculates the bounding box (also known as envelope and extent) of the catchment.<br>

In [ ]:
#########################################################
## Variables
## Note dependency of variables [sub_project_folder, gageid] defined above 

#########################################################
## Commands
w.get_NHDStreamflowGageIdentifiersAndLocation(w.sub_project_folder,w.gageid)
w.get_CatchmentShapefileForNHDStreamflowGage(w.sub_project_folder)
w.get_BoundingboxFromStudyareaShapefile(w.sub_project_folder)

print 'Finished Step'

Using the generated bounding box and user defined start/end dates for RHESSys simulation period, an Essential Terrestrial Variable (***ETV***) RHESSys data bundle (with climate forcing, soils, elevation) are retrieved from the [HydroTerre](http://www.hydroterre.psu.edu) cyberinfrastructure services using the function `HydroTerre_RHESSys_ByExtent`. The zipped data bundle is unzipped for the remaining workflow steps. Please note, the earliest start date is 1979-01-01 and the latest end date is 2009-12-31 for climate forcing (defined at the first step).

In [ ]:
#########################################################
## Variables
## Note dependency of variables [sub_project_folder, ht_start_date, ht_end_date] defined above 

datafolder = '/RHESSys_ETV'
zipfolder = w.sub_project_folder + datafolder

#########################################################
## Commands
extent = w.get_Extent_from_RHESSysWorkflows_Metadata_File()
w.HydroTerre_RHESSys_ByExtent(extent, w.ht_start_date, w.ht_end_date, w.sub_project_folder)

w.create_path(zipfolder)
## Note the zip file name is a constant
zipfilepathname = w.sub_project_folder + '/RHESSys_ETV_Data.zip'
w.unzip_etv_zip_file_at_path(zipfilepathname, zipfolder)

print 'Finished Step'

## Retrieve DEM

The function **`get_USGSDEMForBoundingbox`** downloads 1/3 arcsecond Digital Elevation Model (***DEM***) data from the 
National Elevation Dataset and NHDPlus hydro-conditioned coverages hosted by U.S. Geological Survey (***USGS***) Web Coverage Service (WCS) interface at the [Center for Integrated Data Analytics group](http://cida.usgs.gov/).

In [ ]:
#########################################################
## Variables
## Note dependency of variables [sub_project_folder] defined above 

#########################################################
## Commands
output = w.get_USGSDEMForBoundingbox(w.sub_project_folder)
print(output)

print 'Finished Step'

## Retrieve Landcover Dataset

 **`get_USGSNLCDForDEMExtent`** retrieves the National Landcover Dataset (***NLCD***) within the watershed extent from data services at [RENCI](http://renci.org/).

In [ ]:
#########################################################
## Variables
## Note dependency of variables [sub_project_folder] defined above 

#########################################################
## Commands
output = w.get_USGSNLCDForDEMExtent(w.sub_project_folder)
print(output)

print 'Finished Step'

## Retrieve Soil
The USDA NRCS provides the [Soil Data Mart](http://sdmdataaccess.nrcs.usda.gov/), a web services-based interface for querying and downloading high-resolution SSURGO soils data. **`get_SSURGOFeaturesForBoundingbox`** processes ***SSURGO*** soil data from the United States Department of Agriculture [USDA](http://sdmdataaccess.nrcs.usda.gov/) and the **`GenerateSoilPropertyRastersFromSSURGO`** function rasterizes these soil attributes for [GRASS](https://grass.osgeo.org/) project manipulation.
<br/>
<br/>
**Note:** Output from get_SSURGOFeaturesForBoundingbox has been piped to output variable. For debugging, uncomment print statement (i.e. remove #).

In [ ]:
#########################################################
## Variables
## Note dependency of variables [sub_project_folder] defined above 

#########################################################
## Commands
output = w.get_SSURGOFeaturesForBoundingbox(w.sub_project_folder)
#print(output)
output = w.GenerateSoilPropertyRastersFromSSURGO(w.sub_project_folder)
print(output)

print 'Finished Step'

## Register LAI
Here the resource or user supplied **LAI** is registered. The [HydroTerre](http://www.hydroterre.psu.edu) RHESSys service generates a Leaf Area Index (***LAI***) raster dataset for each month, users can choose which month they are interested in by modifying the **`w.lai_fullpathname_with_ext`** variable. The **`Register_LAI_Raster`** function registers the LAI raster dataset for [GRASS](https://grass.osgeo.org/) project manipulation. The [GRASS](https://grass.osgeo.org/) **`CreateGRASSLocationFromDEM`** setups up the GRASS environment and imports the DEM raster dataset.
<br/> <br/>
&#9971; **Note: ** Change the `lai_name` variable below to match the LAI raster dataset within your resource. Skip this step, if your resource has no LAI.

In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder] defined above 

w.lai_fullpathname_with_ext = zipfolder + '/RHESSys_LAI/LAI_Month0.tif'

#########################################################
## Commands
output = w.Register_LAI_Raster(w.sub_project_folder, w.lai_fullpathname_with_ext, w.publisher)
print(output)

print 'Finished Step'

## Create a new GRASS GIS Location

RHESSys requires that all spatial data used to create a world file and flow table for a RHESSys model be stored in a GRASS GIS mapset. We'll start building these data in RHESSysWorkflows by creating a new GRASS location and mapset within our project directory, and importing our DEM into this mapset using the **CreateGRASSLocationFromDEM** tool. <br/>
<br/> <br/>
&#9971; **Note: ** Change the `description` variable below to match your resource description.

In [ ]:
#########################################################
## Variables
description = '"RHESSys model for Dead Run 5 watershed near Catonsville, MD"'

#########################################################
## Commands
output = w.CreateGRASSLocationFromDEM(w.sub_project_folder, description)
#print(output)

print 'Finished Step'

## Import RHESSys Code
To create worldfiles and flow tables RHESSysWorkflows needs a copy of RHESSys source code. RHESSysWorkflows also uses the new RHESSys ParamDB database and Python libraries to generate vegetation, soil, land use and other parameters needed by RHESSys. RHESSysWorkflows is only compatible with the pre-release version of RHESSys 5.16 and later versions of the code. At present, and for first-time users, the most reliable way to import ParamDB and RHESSys source code into your project is to download the code from GitHub using the **ImportRHESSysSource** tool. However, this tool is also capable of importing RHESSys source code stored on your computer. This allows you to import the code from a previous RHESSysWorkflows project; ParamDB is always downloaded from GitHub, even when RHESSys source code is imported from a local source.

The `ImportRHESSysSource` function downloads and compiles the latest [RHESSys](https://github.com/RHESSys/RHESSys) source code in the user defined project folder. This make take a couple minutes.     

In [ ]:
#########################################################
## Commands

output = w.ImportRHESSysSource(w.sub_project_folder)
#print(output)

print 'Finished Step'

## Import Climate Data

Because of the great variability of climate data formats, and the complexity of time-series workflows, we have chosen to focus development effort on RHESSysWorkflows toward making it easier to acquire and manipulate geospatial data required for building RHESSys work files and flow tables. This means that the modeler is responsible for building the climate data necessary for building RHESSys world files and performing model runs. <br>
<br>
The [HydroTerre](http://www.hydroterre.psu.edu) RHESSys service generates climate data within the user defined start and end period. Users can download data from other services (i.e. [HydroShare](https://www.hydroshare.org/)) and modify the `climate_data_fullpathname` variable. The `ImportClimateData` function imports the ***climate*** data already in RHESSys file formats into the [RHESSys](https://github.com/RHESSys/RHESSys) project directory. If the user is using climate station data, the `station_data_fullpathname` variable should be defined. <br>
<br>
RHESSysWorkflows provides the **ImportClimateData** tool to import RHESSys climate data into your project.<br>


In [ ]:
#########################################################
## Variables
## Note dependency of variables [zipfolder] defined above 
climate_name = 'RHESSys_Climate'

climate_file = zipfolder + '/' + climate_name

w.climate_data_fullpathname = climate_file;
w.station_data_fullpathname = w.climate_data_fullpathname
#########################################################
## Commands
output = w.ImportClimateData(w.sub_project_folder, w.climate_data_fullpathname)
print(output)

print 'Finished Step'

## Delineate Watershed
RHESSysWorkflows automates the process of delineating your study watershed based on the location of the streamflow gage registered in the workflow using EcohydroLib. As part of this process, many datasets needed by RHESSys will be derived from the DEM. To delineate the watershed we use the **DelineateWatershed** tool.

Here the **cell_size** parameter specifies the minimum size (in DEM cells) for subwatersheds generated by the GRASS command r.watershed.

The **area_estimate** parameter allows you to provide a guess of the area (in sq. km) of the delineated watershed. DelineateWatershed will report whether the watershed is within 20% of the area. You can view the delineated watershed in GRASS by displaying the raster map named basin. If the area or the shape of the delineated watershed differs greatly from what you expect, you may need to vary how DelineateWatershed snaps your streamflow gage onto the stream network. This is accomplished by either changing the -s (a.k.a. --streamThreshold) or stream threshold parameter and/or the -w (a.k.a. --streamWindow) parameter passed to r.findtheriver.
<br/> <br/>
&#9971; **Note: ** Change the `cell_size` and `area_estimate` variables below to match your resource watershed property. Uncomment the `print(output)` statement to evaluate the area estimate.

In [ ]:
#########################################################
## Variables

cell_size = 250
area_estimate = 1.62

#########################################################
## Commands
output = w.DelineateWatershed(w.sub_project_folder, cell_size, area_estimate)
#print(output)

print 'Finished Step'

## Generate Patch Map

RHESSysWorkflows provides **GeneratePatchMap**, an automated tool for creating gridded and clumped patch maps. Gridded patch maps consist of a regular grid of the same resolution and extent as the DEM; clumped maps can be created using elevation or topographic wetness index rasters. Modelers can also use custom patch maps registered via EcohydroLib's RegisterRaster tool and imported into GRASS using ImportRasterMapIntoGRASS. <br>
<br>
<br>
**Options** include the following: <br>
**-t {grid,clump}**             Type of patch to be generated: uniform grid or clumps based on elevation. <br>
**-c {elevation,wetness_index}**             Type of patch to be generated: uniform grid or clumps based on elevation.<br>
**-z**             Use patch map as zone map even if zone map is already defined. By default if a zone map is present, this script will not set the patch map as the zone map.<br>
**--overwrite**           Overwrite existing datasets in the GRASS mapset. If not specified, program will halt if a dataset already exists.<br> 

In [ ]:
#########################################################
## Variables
grid_options =  " -t clump -c elevation"

patch_options = " -p " + w.sub_project_folder + grid_options

#########################################################
## Commands
output = w.GeneratePatchMapOptions(patch_options)
#print(output)

print 'Finished Step'

## Process Soil Texture Map

Since we used EcohydroLib's SSURGO tools to generate percent sand and percent clay raster maps for our watershed, we can use the GRASS add-on r.soils.texture to generate USDA soil texture classes, for which RHESSys's ParamDB contains parameters. It is also possible to use custom soil maps (refer to the Tutorial).

To generate our soil texture map in GRASS, as well as the corresponding RHESSys soil definition files, use the **GenerateSoilTextureMap** tool as follows: <br>
<br>
<br>
**Options** include the following: <br>
**--defonly**             Only generate soil definition files, do not try to create soil texture map. Map must already exist. <br>
**--overwrite**           Overwrite existing datasets in the GRASS mapset. If not specified, program will halt if a dataset already exists.<br>

In [ ]:
#########################################################
## Variables
soil_options =  " --overwrite"

#########################################################
## Commands
output = w.GenerateSoilTextureMap(w.sub_project_folder, options=soil_options)
#print(output)

print 'Finished Step'

## Generate derived landcover maps

Now we generating LAI, isohyet, roads and landcover maps from your project directory into GRASS using the **ImportRasterMapIntoGRASS** tool. <br>
**--type** {all,landcover,roof_connectivity,soil,lai,patch,zone,isohyet,leafc,rootdepth,roads,stream_burned_dem}  <br>
**--method** {none,nearest,bilinear,bicubic} <br>
<br/>

In [ ]:
#########################################################
## Variables
raster_type = 'lai'
raster_sample = 'nearest'

#########################################################
## Commands
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, raster_type, raster_sample)
#print(output)

print 'Finished Step'

In [ ]:
#########################################################
## Variables
raster_type = 'landcover'
raster_sample = 'nearest'

#########################################################
## Commands
output = w.ImportRasterMapIntoGRASS(w.sub_project_folder, raster_type, raster_sample)
#print(output)

print 'Finished Step'

## Register landcover Rules

The RegisterLandcoverReclassRules function generates landcover raster map reclassification rules ( stratum, landuse, impervious, road, lai-recode ) which are then used by the GenerateLandcoverMaps function to generate derived landscover data for RHESSys.


In [ ]:
#########################################################
## Commands
output = w.RegisterLandcoverReclassRules(w.sub_project_folder)
#print(output)

print 'Finished Step'

Once the landcover reclass rules are in place, it is very easy to generate the raster maps derived from the landcover data as well as the vegetation and land use definition files needed by RHESSys using ***GenerateLandcoverMapsOptions*** tool.
<br>
<br>
**Options** include the following: <br>
**-l, --makeLaiMap**      Make LAI map <br>
**--skipRoads**           Do not make roads map <br>
**--defonly**             Only generate landuse and stratum definition files, do not try to create maps. Maps must already exist. <br>
**--overwrite**           Overwrite existing datasets in the GRASS mapset. If not specified, program will halt if a dataset already exists.<br>
<br/> <br/>
&#9971; **Note: ** Modify **landcover_options** below to match your data inputs.


In [ ]:
#########################################################
## Variables
landcover_options =  " --overwrite"

options = " -p " + w.sub_project_folder + landcover_options

#########################################################
## Commands
output = w.GenerateLandcoverMapsOptions(options)
#print(output)

print 'Finished Step'

## Generate World Template and World File
Now we are almost ready to create the world file for our watershed. First we must create the template from which the world file will be created. To do this, we'll use the **GenerateWorldTemplate** tool. Fortunately this is very easy because the metadata store contains nearly all the information needed to create the template.

**Options** include the following: <br>
**-c (a.k.a. --climateStation)**      The climate station to associate with the worldfile. Must be one of the climate stations specified in the "climate_stations" key in the "rhessys" section of the metadata<br>
**--aspectMinSlopeOne**           Use slope map with a minimum value of 1.0 to be used for calculating spherical average aspect. Needed for areas of low slope due to limitations of RHESSys grass2world, which truncates slopes <1 to 0.0, which causes spherical average of <br> <br/>
**-v, --verbose **        Print detailed information about what the program is doing<br>

<br/> <br/>
&#9971; **Note: ** Modify **world_template_options** below to match your climate data inputs.

In [ ]:
#########################################################
## Variables
world_template_flags =  " -c HT_RHESSys"
world_template_options = " -p " + w.sub_project_folder + world_template_flags

#########################################################
## Commands
output = w.GenerateWorldTemplateOptions(world_template_options)
print(output)

print 'Finished Step'

Now use the **CreateWorldfile** tool to create a world file using this template. <br/>
Note: Output from CreateWorldfile has been piped to output variable. For debugging, uncomment print statement (i.e. remove #).

In [ ]:
#########################################################
## Commands
output = w.CreateWorldfile(w.sub_project_folder)
#print(output)

print 'Finished Step'

## Create Flow Table
As with worldfile creation, at this point in the workflow, RHESSysWorkflows's metadata store contains all the information needed to create a flow table using the createflowpaths (CF) RHESSys program. We'll use CreateFlowtable to create our flow table.
<br><br>
This will result in the creation of a flow table called world.flow in the flow directory of your rhessys directory. Now we have almost everything we need to run RHESSys simulations.
<br><br>
See the RHESSysWorkflows tutorial to learn how to [route surface flows for road pixels directly to the stream](https://github.com/selimnairb/RHESSysWorkflows#creating-the-flow-table) and [create surface flow tables using a roof connectivity map](https://github.com/selimnairb/RHESSysWorkflows#creating-a-surface-flow-table-using-a-roof-connectivity-map)
<br>
<br>
**Options** include the following: <br>
**--routeRoads**      Tell createflowpaths to route flow from roads to the nearest stream pixel (requires roads_rast to be defined in metadata)<br>
**--routeRoofs** Tell createflowpaths to route flow from roof tops based on roof top connectivity to nearest impervious surface
(requires roof_connectivity_rast and impervious_rast to be defined in metadata)<br>
**-f, --force ** Run createflowpaths even if DEM x resolution does not match y resolution<br>
**--ignoreBurnedDEM ** Ignore stream burned DEM, if present. Default DEM raster will be used for all operations. If not specified and if stream burned raster is present, stream burned DEM will be used for generating the flow table.<br>
**-v, --verbose **        Print detailed information about what the program is doing<br>


<br/> <br/>
&#9971; **Note: ** Modify **flow_table_options** below to match your data inputs.

In [ ]:
#########################################################
## Variables
flow_table_flags = " --routeRoads -v"
flow_table_options = " -p " + w.sub_project_folder + flow_table_flags

#########################################################
## Commands
output = w.CreateFlowtableOptions(flow_table_options)
print(output)

print 'Finished Step'

## Initializing vegetation carbon and nitrogen stores
RHESSys provides a program called LAIread to initialize vegetation carbon and nitrogen stores in your world file.<br>
<br>
Initializing carbon and nitrogen stores is a multi-step process that involves running LAI read to generate a redefine worldfile, running a 3-day RHESSys simulation to incorporate the redefine worldfile, writing out a new worldfile with initialized vegetation carbon and nitrogen stores. RHESSysWorkflows automates all of these processes for you using the tool RunLAIRead, which can even figure out what date to start the 3-day RHESSys simulation on based on your climate data. <br>
<br>
### Note: The output below is long, however, it is recommended to scroll to the tail end of the output to check for error messages.

In [ ]:
#########################################################
## Commands
output = w.RunLAIRead(w.sub_project_folder)
print(output)

print 'Finished Step'

## Creating a RHESSys TEC file
We need one more thing before we can run our model, a TEC file. TEC stands for "temporal event controller". We use a TEC file to tell RHESSys to do things on at certain times during a simulation.

In [ ]:
#########################################################
## Variables
model_start = str(w.model_start_year) + ' ' + str(w.model_start_month) + ' ' + str(w.model_start_day) + ' 1'

tec_file = "tec_daily.txt"
tec_output_location = w.sub_project_folder + "/rhessys/tecfiles/" + tec_file
tec_cmd = ' echo "' + model_start + ' print_daily_on" > '
tec_flags = tec_cmd + tec_output_location
tec_options = " -p " + w.sub_project_folder + tec_flags

#########################################################
## Commands
output = w.RunCmdOptions(tec_options)
print(output)

print 'Finished Step'


## Running RHESSys models
Once you have built a RHESSys model using RHESSysWorkflows, you are free to run your model manually. However this will not capture information about model runs in your project metadata. If you would like to record your runs in your project metadata, use the **RunModel** command.


Because the project metadata knows where RHESSys is installed in your project directory, you don't have to specify the full path of any of the RHESSys input files (e.g. world files, flow tables, TEC files, etc), you only need to specify the filenames. RunModel will echo RHESSys console outlet to the screen (if the -v or verbose option is specified as above), and will always save the same output into a file named 'rhessys.out' stored in the output folder for each model run. The output folder will be named based on the value you provide for the '-pre' or output prefix option. Model output is stored in the output directory of the rhessys directory of your project directory.

For more details on using RunModel see the RHESSysWorkflows [tutorial](https://github.com/selimnairb/RHESSysWorkflows#running-rhessys-models).
<br>
<br>
**Options** include the following: <br>
**-d DESCRIPTION, --description** Description of the model run<br>
**-pre OUTPUTPREFIX** Filename prefix to use for output files, relative to output directory in the RHESSys directory of the project<br>
**-st STARTDATE STARTDATE STARTDATE STARTDATE** Start date and time of the model run, of the form "YYYY M D H"<br>
**-ed ENDDATE ENDDATE ENDDATE ENDDATE** Date date and time of the model run, of the form "YYYY M D H".<br>
**-w WORLDFILE** Filename of the tecfile to use for the model run, specified relative to the tec directory in the RHESSys directory of the projec<br>
**-t TECFILE **        Filename of the tecfile to use for the model run, specified relative to the tec directory in the RHESSys directory of the project<br>
**-r [FLOWTABLES [FLOWTABLES ...]]**Filename(s) of the flow table(s) to use for the model run, specified relative to the flowtable directory in the RHESSys directory of the project. If one flow table is supplied, it will be used for subsurface and surface routing. If two flow tables are supplied the first will be use for subsurface routing, the second for surface<br>
**--basin ** Tell RHESSys to output at the basin spatial level<br>
**--hillslope ** Tell RHESSys to output at the hillslope spatial level<br>
**--zone ** Tell RHESSys to output at the zone spatial level<br>
**--patch ** Tell RHESSys to output at the patch spatial level<br>
**--canopy ** Tell RHESSys to output at the canopy stratum spatial level<br>
**-v, --verbose **        Print detailed information about what the program is doing<br>


<br/> <br/>
&#9971; **Note: ** Modify **options** below to match your data inputs.

In [ ]:
#########################################################
## Variables
## Note dependency of variables [w.model_start_year, w.model_start_month, w.model_start_day] defined above 
## Note dependency of variables [w.model_end_year, w.model_end_month, w.model_end_day] defined above 
## Note dependency of variables [tec_file] defined above 

model_start = str(w.model_start_year) + ' ' + str(w.model_start_month) + ' ' + str(w.model_start_day) + ' 1'
model_end = str(w.model_end_year) + ' ' + str(w.model_end_month) + ' ' + str(w.model_end_day) + ' 1'

description = '"Test model run"'  ##Must use double quotes here
spatial_level = "--basin"
prefix = "test"
world_file = "world"
tec_file = "tec_daily.txt" ##Defined above step
flow_table = "world.flow"
s_flag = '0.07041256017 133.552915269 1.81282283058'
sv_flag = '4.12459677088 78.3440566535'
gw_flag = '0.00736592779294 0.340346799457'
flow_table_flag = ' -r ' + flow_table + ' -- -s ' + s_flag + ' -sv ' + sv_flag + ' -gw ' + gw_flag

runmodel_flags = '  -d ' + description + ' ' + spatial_level + ' -pre ' + prefix +' -st ' + model_start + ' -ed ' + model_end + ' -w ' + world_file + ' -t ' + tec_file + flow_table_flag
runmodel_options = " -v -p " + w.sub_project_folder + runmodel_flags
##print(runmodel_options)

#########################################################
## Commands
output = w.RunModelOptions(runmodel_options)
print(output)

print 'Finished Step'


## Plot RHESSys model Results

To aid in analyzing the RHESSys simulation results, the `plot_rhessys_results` function plots model results (***data ***) against user supplied observed data (***obs_data***).


As part of this tutorial, we supply a sample data observation file. Users can either modify the url path below to download data from a data service, or, upload data via the Jupyter tree, by clicking on the jupyter icon (top left of web page). 

In [ ]:
#########################################################
## Variables

url_path = "http://gidesigner.renci.org/data/DR5_discharge_WY2008-2012.txt"

data_folder = '/home/jovyan/work/notebooks/data'
obs_data =  os.path.join(data_folder, 'DR5_discharge_WY2008-2012.txt')

#########################################################
## Commands
wget.download(url_path, out=data_folder)

print 'Finished Step'

In [ ]:
#########################################################
## Variables
model_output = 'rhessys/output/test/rhessys_basin.daily'
outfileSuffix = 'test_plot'
data_column = 'streamflow'
plot_title = 'DR5 streamflow'
legend = 'Test simulation'
y_axis_title = 'Streamflow (mm/day)'

#########################################################
## Commands
data =  os.path.join(w.sub_project_folder, model_output)

plot_rhessys_results(
    outfileSuffix = outfileSuffix, 
    obs = obs_data, 
    column = data_column,
    data = [data], 
    legend = [legend],
    title = plot_title,
    y = y_axis_title)

print 'Finished Step'

## Help with error messages

If you see an error message, such as **returned non-zero exit status 1** check the log file located in the **w.sub_project_folder** folder. The log file name is the same name as the project folder, with **.log** appended.

## Known Limitations and Future Additions

* We are ***missing user parameters*** to control the RHESSys simulation. For example, different ways to generate patches. As well as controlling the data workflows, such as metadata for `CreateGRASSLocationFromDEM`.
* The ***areaEstimate*** input in `DelineateWatershed` function requires automation. The area returned by the HydroTerre RHESSys workflow uses USGS HUC-12s that are often larger than the watershed generated by the `DelineateWatershed` function.
* New tools are required to process the ***LAI*** raster datasets for RHESSys simulations.
* The RHESSysWorkflows do allow users to specify and/or upload ***climate station data***. We envision a new web-service that can automatically detect the climate stations within the watershed and prepare the data. Here, we are using climate data from HydroTerre which has one climate normal from 1979-01-01 to 2009-12-31. Users interested in data from the year 2010 to present will be required to upload RHESSys climate files.
* The ***ETV***  bundle has additional data not used by this workflow and future tools will allow users the option where to retrieve data services from with graphic user interface (GUI) control.
* This notebook retrieves RHESSys code from the github repository. Future notebooks will empower users to upload their code version and/or import specific model versions.

## Resources

* RHESSys
  * [Setup](http://fiesta.bren.ucsb.edu/~rhessys/setup/setup.html)
  * [Wiki](http://fiesta.bren.ucsb.edu/~rhessys/)
* Data
  * [HydroShare](https://www.hydroshare.org/)
  * [USGS Data and Tools](https://www.usgs.gov/products/data-and-tools/data-and-tools-topics)
  * [USDA Data gateway](https://gdg.sc.egov.usda.gov/)
  * [HydroTerre](http://hydroterre.psu.edu/)
  


## Funding
This work was supported by the following NSF grants

    Award no. 1239678 EAGER: Collaborative Research: Interoperability Testbed-Assessing a Layered Architecture for Integration of Existing Capabilities

    Award no. 0940841 DataNet Federation Consortium.

    Award no. 1148090 Collaborative Research: SI2-SSI: An Interactive Software Infrastructure for Sustaining Collaborative Innovation in the Hydrologic Sciences


## Credits

**Lawrence E. Band** lband@email.unc.edu <br/>
**Brian Miles** brian_miles@unc.edu <br/> 
**Laurence Lin** laurence.lin@icloud.com <br/>
**John Duncan** jon.m.duncan@gmail.com <br/>
**Lorne Leonard** lnl3@psu.edu <br/>
